In [1]:
info = """
LangChain 1.0 - Agent 执行循环（ReAct 模式）
============================================

本模块重点讲解：
1. ReAct 执行循环的详细过程（Reason → Act → Observe）
2. 流式输出（streaming）
3. 查看中间步骤
4. 理解消息流转

⚠️ API 说明：
- 本模块使用 `create_react_agent`（来自 langgraph.prebuilt）
- 这是 LangGraph 1.0 的预构建图，明确使用 ReAct 架构
- 适合学习 Agent 执行循环的底层原理

🔄 ReAct 循环：
   Reason（推理）→ Act（行动）→ Observe（观察）→ 循环直到完成
"""
print(info)


LangChain 1.0 - Agent 执行循环（ReAct 模式）

本模块重点讲解：
1. ReAct 执行循环的详细过程（Reason → Act → Observe）
2. 流式输出（streaming）
3. 查看中间步骤
4. 理解消息流转

⚠️ API 说明：
- 本模块使用 `create_react_agent`（来自 langgraph.prebuilt）
- 这是 LangGraph 1.0 的预构建图，明确使用 ReAct 架构
- 适合学习 Agent 执行循环的底层原理

🔄 ReAct 循环：
   Reason（推理）→ Act（行动）→ Observe（观察）→ 循环直到完成



In [2]:
import os
import sys

In [5]:
# 添加父目录到路径以导入工具
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, os.path.join(parent_dir, '04_custom_tools', 'tools'))


In [6]:
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langchain.agents import create_agent

In [7]:
from calculator import calculator
from weather import get_weather

In [8]:
# 加载环境变量
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

if not GROQ_API_KEY or GROQ_API_KEY == "your_groq_api_key_here":
    raise ValueError(
        "\n请先在 .env 文件中设置有效的 GROQ_API_KEY\n"
        "访问 https://console.groq.com/keys 获取免费密钥"
    )

In [9]:
model = init_chat_model("groq:llama-3.3-70b-versatile", api_key=GROQ_API_KEY)


In [10]:
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x111f89400>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x111f8a270>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [11]:
print(    """
    示例1：查看 Agent 执行循环的每一步

    关键：response['messages'] 包含完整的对话历史
    """)


    示例1：查看 Agent 执行循环的每一步

    关键：response['messages'] 包含完整的对话历史
    


In [12]:
print("\n" + "="*70)
print("示例 1：Agent 执行循环详解")
print("="*70)


示例 1：Agent 执行循环详解


In [13]:
agent = create_agent(
    model=model,
    tools=[calculator],
    system_prompt="你是一个智能助手，请根据用户指令执行任务",
)

In [16]:
response = agent.invoke(
    {"messages": [{"role": "user", "content": "计算 123 乘以 6 的结果是多少？"}]},
)

In [17]:
response

{'messages': [HumanMessage(content='计算 123 乘以 6 的结果是多少？', additional_kwargs={}, response_metadata={}, id='6c84b987-be22-4e34-971b-7c23640e4a98'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '0t0k7etfy', 'function': {'arguments': '{"a":123,"b":6,"operation":"multiply"}', 'name': 'calculator'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 364, 'total_tokens': 389, 'completion_time': 0.050520347, 'completion_tokens_details': None, 'prompt_time': 0.035240815, 'prompt_tokens_details': None, 'queue_time': 0.054967075, 'total_time': 0.085761162}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_68f543a7cc', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b3ac8-5761-70e1-b50c-d5daed1b235b-0', tool_calls=[{'name': 'calculator', 'args': {'a': 123, 'b': 6, 'operation': 'multiply'}, 'id': '0t0k7etfy', 'type': 'tool_call'}], usage_meta

In [20]:
print("\n完整消息历史：")
for i ,msg in enumerate(response["messages"],1):
    print(f"\n{'='*60}")
    print(f"消息 {i}: {msg.__class__.__name__}")
    print(f"{'='*60}")
    if hasattr(msg, 'content') and msg.content:
        print(f"内容:\n{msg.content}")
    if hasattr(msg, 'tool_calls') and msg.tool_calls:
        print(f"工具调用:\n{msg.tool_calls}")
        for tc in msg.tool_calls:
            print(f"  工具名称: {tc['name']}")
            print(f"  参数: {tc['args']}")


完整消息历史：

消息 1: HumanMessage
内容:
计算 123 乘以 6 的结果是多少？

消息 2: AIMessage
工具调用:
[{'name': 'calculator', 'args': {'a': 123, 'b': 6, 'operation': 'multiply'}, 'id': '0t0k7etfy', 'type': 'tool_call'}]
  工具名称: calculator
  参数: {'a': 123, 'b': 6, 'operation': 'multiply'}

消息 3: ToolMessage
内容:
123.0 multiply 6.0 = 738.0

消息 4: AIMessage
内容:
计算结果是 738。


In [21]:
print("\n\n执行流程：")
print("""
1. HumanMessage    → 用户问题
2. AIMessage       → AI 决定调用工具（包含 tool_calls）
3. ToolMessage     → 工具执行结果
4. AIMessage       → AI 基于结果生成最终答案
""")

print("\n关键点：")
print("  - Agent 自动完成这个循环")
print("  - 所有步骤都记录在 messages 中")
print("  - 最后一条消息是最终答案")



执行流程：

1. HumanMessage    → 用户问题
2. AIMessage       → AI 决定调用工具（包含 tool_calls）
3. ToolMessage     → 工具执行结果
4. AIMessage       → AI 基于结果生成最终答案


关键点：
  - Agent 自动完成这个循环
  - 所有步骤都记录在 messages 中
  - 最后一条消息是最终答案


In [22]:
"""
    示例2：实时查看 Agent 的输出

    使用 .stream() 方法
    """
print("\n" + "="*70)
print("示例 2：流式输出")
print("="*70)


示例 2：流式输出


In [24]:
agent = create_agent(
        model=model,
        tools=[calculator, get_weather],
    system_prompt="你是一个有帮助的助手。"
    )

print("\n问题：北京天气如何？然后计算 10 加 20")
print("\n流式输出（实时显示）：")
print("-" * 70)


问题：北京天气如何？然后计算 10 加 20

流式输出（实时显示）：
----------------------------------------------------------------------


In [36]:
for chunk in agent.stream({"messages":[{
"role": "user", "content": "北京的天气如何？然后计算 10 加 20"
}]}):
    print(chunk, end='\n', flush=True)
    # if "model" in chunk or "tools" in chunk:
        

{'model': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'pn7gzpy99', 'function': {'arguments': '{"city":"北京"}', 'name': 'get_weather'}, 'type': 'function'}, {'id': '0e1fsg5hg', 'function': {'arguments': '{"a":10,"b":20,"operation":"add"}', 'name': 'calculator'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 466, 'total_tokens': 504, 'completion_time': 0.076976267, 'completion_tokens_details': None, 'prompt_time': 0.04605711, 'prompt_tokens_details': None, 'queue_time': 0.058363719, 'total_time': 0.123033377}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_68f543a7cc', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b3ada-1152-71d2-9c47-c3d7a8bbbe04-0', tool_calls=[{'name': 'get_weather', 'args': {'city': '北京'}, 'id': 'pn7gzpy99', 'type': 'tool_call'}, {'name': 'calculator', 'args': {'a': 10, 'b': 20, 'operation'

In [37]:
"""
    示例3：Agent 执行多个工具调用

    理解复杂任务的执行过程
    """
print("\n" + "="*70)
print("示例 3：多步骤执行")
print("="*70)


示例 3：多步骤执行


In [40]:
agent = create_agent(
    model=model,
    tools=[calculator, get_weather],
    system_prompt="你是一个智能助手，请根据用户指令执行任务",
    
)

In [41]:
response = agent.invoke({
    "messages": [{
        "role": "user",
        "content": "告诉我今天北京的天气，然后计算 15% 的 250 是多少？最后再乘以 4。"
    }]
})

In [42]:
tools_calls_count = 0


In [46]:
for msg in response["messages"]:
    if hasattr(msg, 'tool_calls') and msg.tool_calls:
        tools_calls_count += len(msg.tool_calls)
        print(f"\nAI回复调用工具的消息：{msg}")


AI回复调用工具的消息：content='' additional_kwargs={'tool_calls': [{'id': 'b7gyfxv59', 'function': {'arguments': '{"city":"北京"}', 'name': 'get_weather'}, 'type': 'function'}, {'id': '05pxes51b', 'function': {'arguments': '{"a":250,"b":0.15,"operation":"multiply"}', 'name': 'calculator'}, 'type': 'function'}, {'id': 'ezqbxxhsy', 'function': {'arguments': '{"a":37.5,"b":4,"operation":"multiply"}', 'name': 'calculator'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 66, 'prompt_tokens': 482, 'total_tokens': 548, 'completion_time': 0.142373785, 'completion_tokens_details': None, 'prompt_time': 0.049505382, 'prompt_tokens_details': None, 'queue_time': 0.058371148, 'total_time': 0.191879167}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_c06d5113ec', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'} id='lc_run--019b3ade-2918-7992-aea4-c17bd5b2450d-0' tool_calls=[{'name': 'get_weather', 'args': {

In [44]:
tools_calls_count

3

In [47]:
print("\n关键点：")
print("  - Agent 可以多次调用工具")
print("  - 每次调用的结果会影响下一步")
print("  - 直到得到最终答案")


关键点：
  - Agent 可以多次调用工具
  - 每次调用的结果会影响下一步
  - 直到得到最终答案


In [48]:
"""
    示例4：在执行过程中查看状态

    使用 stream 并检查每个 chunk
    """

'\n    示例4：在执行过程中查看状态\n\n    使用 stream 并检查每个 chunk\n    '

In [51]:
print("\n" + "="*70)
print("示例 4：查看中间状态")
print("="*70)


示例 4：查看中间状态


In [52]:
agent = create_agent(
    model=model,
    tools=[calculator],
system_prompt="你是一个有帮助的助手。"
)

print("\n问题：100 除以 5 等于多少？")
print("\n执行步骤：")


问题：100 除以 5 等于多少？

执行步骤：


In [62]:
step = 0

for chunk in agent.stream({
        "messages": [{"role": "user", "content": "100 除以 5 等于多少？"}]
    }):
        step += 1
        print(f"\n步骤 {step}:")

        print(chunk, end='', flush=True)
        


步骤 1:
{'model': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'epfkzv1vc', 'function': {'arguments': '{"a":100,"b":5,"operation":"divide"}', 'name': 'calculator'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 357, 'total_tokens': 382, 'completion_time': 0.067246293, 'completion_tokens_details': None, 'prompt_time': 0.01989956, 'prompt_tokens_details': None, 'queue_time': 0.06226966, 'total_time': 0.087145853}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_68f543a7cc', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b3aec-8d97-7a91-9bda-c6590bea60e8-0', tool_calls=[{'name': 'calculator', 'args': {'a': 100, 'b': 5, 'operation': 'divide'}, 'id': 'epfkzv1vc', 'type': 'tool_call'}], usage_metadata={'input_tokens': 357, 'output_tokens': 25, 'total_tokens': 382})]}}
步骤 2:
{'tools': {'messages': [ToolMessage(content=

In [63]:
"""
    示例5：详解各种消息类型

    Agent 执行循环中的消息类型
    """

'\n    示例5：详解各种消息类型\n\n    Agent 执行循环中的消息类型\n    '

In [64]:
print("\n" + "="*70)
print("示例 5：消息类型详解")
print("="*70)



示例 5：消息类型详解


In [68]:
agent = create_agent(
        model=model,
        tools=[get_weather],
    system_prompt="你是一个有帮助的助手。"
    )

In [69]:
response = agent.invoke({
        "messages": [{"role": "user", "content": "上海天气如何？"}]
    })

print("\n消息类型分析：")
for msg in response['messages']:
    msg_type = msg.__class__.__name__

    if msg_type == "HumanMessage":
        print(f"\n[HumanMessage] 用户输入")
        print(f"  内容: {msg.content}")

    elif msg_type == "AIMessage":
        if hasattr(msg, 'tool_calls') and msg.tool_calls:
            print(f"\n[AIMessage] AI 决定调用工具")
            print(f"  工具: {msg.tool_calls[0]['name']}")
            print(f"  参数: {msg.tool_calls[0]['args']}")
        else:
            print(f"\n[AIMessage] AI 的最终回答")
            print(f"  内容: {msg.content}")

    elif msg_type == "ToolMessage":
        print(f"\n[ToolMessage] 工具执行结果")
        print(f"  工具: {msg.name}")
        print(f"  结果: {msg.content}")


消息类型分析：

[HumanMessage] 用户输入
  内容: 上海天气如何？

[AIMessage] AI 决定调用工具
  工具: get_weather
  参数: {'city': '上海'}

[ToolMessage] 工具执行结果
  工具: get_weather
  结果: 多云，温度 18°C，有轻微雾霾

[AIMessage] AI 的最终回答
  内容: 上海的天气是多云，温度18°C，有轻微雾霾。


In [70]:
"""
    示例6：使用执行循环的最佳实践
    """
print("\n" + "="*70)
print("示例 6：执行循环最佳实践")
print("="*70)



示例 6：执行循环最佳实践


In [71]:
print("""
最佳实践：

1. 获取最终答案
   final_answer = response['messages'][-1].content

2. 查看是否使用了工具
   used_tools = [
       msg.tool_calls[0]['name']
       for msg in response['messages']
       if hasattr(msg, 'tool_calls') and msg.tool_calls
   ]

3. 流式输出用于用户体验
   for chunk in agent.stream(input):
       # 实时显示给用户

4. 调试时查看完整历史
   for msg in response['messages']:
       print(msg)

5. 错误处理
   try:
       response = agent.invoke(input)
   except Exception as e:
       print(f"Agent 执行错误: {e}")

6. API 注意事项（LangChain 1.0）
   - 使用 create_agent（LangChain 1.0 新 API）
   - LangChain 1.0 新 API
    """)


最佳实践：

1. 获取最终答案
   final_answer = response['messages'][-1].content

2. 查看是否使用了工具
   used_tools = [
       msg.tool_calls[0]['name']
       for msg in response['messages']
       if hasattr(msg, 'tool_calls') and msg.tool_calls
   ]

3. 流式输出用于用户体验
   for chunk in agent.stream(input):
       # 实时显示给用户

4. 调试时查看完整历史
   for msg in response['messages']:
       print(msg)

5. 错误处理
   try:
       response = agent.invoke(input)
   except Exception as e:
       print(f"Agent 执行错误: {e}")

6. API 注意事项（LangChain 1.0）
   - 使用 create_agent（LangChain 1.0 新 API）
   - LangChain 1.0 新 API
    
